In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import pytrade
import matplotlib.pyplot as plt
import datetime

pd.options.display.float_format = '{:,.3f}'.format

In [2]:
%%time
raw = pytrade.get_data(reload = True)
ratios = pytrade.calculate_ratio(raw["Adj Close"])
output = pytrade.analysis_1.run(ratios)
output.sort_values(by="short_term_metric",ascending=False,inplace=True)
my_shortterm = output.loc[["SGEN","DXCM","BMRN","JD","MSFT"]]
my_longterm = output.loc[["AMGN","AAPL","UNH","BMY","NLOK","OKE","INTC","KLAC","REGN"]]
shortterm_portfolio = pytrade.Portfolio("st_portfolio", raw)
longterm_portfolio = pytrade.Portfolio.load("lt_portfolio", raw)
output

[*********************100%***********************]  537 of 537 completed
Wall time: 1min 9s


,latest,three_wk_high,three_wk_low,three_m_high,three_m_low,one_yr_high,one_yr_low,three_wk_range,three_m_range,one_yr_range,short_term_stochastic,long_term_stochastic,st_potential,lt_potential,short_term_metric,long_term_metric
NTES,450.280,493.439,448.722,493.439,368.893,493.439,229.545,0.035,0.653,0.836,0.802,0.183,0.096,0.096,1.000,0.409
BMRN,122.830,130.955,122.830,132.650,100.190,132.650,70.887,0.000,0.697,0.841,0.841,0.144,0.064,0.080,0.974,0.332
CMG,"1,112.230","1,164.437","1,110.237","1,164.437",966.065,"1,164.437",624.349,0.037,0.737,0.903,0.867,0.167,0.047,0.047,0.964,0.277
EA,134.780,142.287,134.640,142.287,118.994,142.287,94.603,0.018,0.678,0.843,0.824,0.165,0.056,0.056,0.939,0.296
MCO,282.660,300.488,282.660,300.488,268.946,300.488,212.761,0.000,0.435,0.797,0.797,0.362,0.063,0.063,0.927,0.511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PVH,50.710,51.306,43.405,69.160,43.405,107.703,40.078,0.925,0.284,0.157,-0.767,-0.126,-0.135,-0.104,-1.048,-0.371
UHS,109.070,109.070,90.476,121.018,90.476,167.335,90.476,1.000,0.609,0.242,-0.758,-0.367,-0.170,-0.170,-1.111,-0.770
UA,10.030,10.030,8.465,10.835,7.766,25.984,7.766,1.000,0.738,0.124,-0.876,-0.613,-0.150,-0.226,-1.185,-1.147
UAA,11.300,11.300,9.373,12.173,8.600,29.234,8.600,1.000,0.756,0.131,-0.869,-0.625,-0.163,-0.239,-1.207,-1.189


In [ ]:
output.sort_values(by="long_term_metric",ascending=False,inplace=True)
output

In [ ]:
my_longterm

In [ ]:
my_shortterm

In [ ]:
plt.close('all')
ratios["HUM"].tail(200).plot()

In [ ]:
longterm_portfolio.report()

In [ ]:
longterm_portfolio.market_current_value()

In [ ]:
shortterm_portfolio.report()

In [ ]:
shortterm_portfolio.market_current_value()

In [ ]:
# shortterm_portfolio.deposit(7174.44, datetime.date(2020, 6, 22))
# shortterm_portfolio.buy("PKI", 73, 7174.44, datetime.date(2020, 6, 22))
# shortterm_portfolio.deposit(787.11, datetime.date(2020, 6, 23))
# shortterm_portfolio.buy("ZBRA", 3, 787.11, datetime.date(2020, 6, 23))
# shortterm_portfolio.deposit(7535.5, datetime.date(2020, 6, 24))
# shortterm_portfolio.buy("QRVO", 70, 7535.5, datetime.date(2020, 6, 24))
# shortterm_portfolio.deposit(5505.65, datetime.date(2020, 7, 2))
# shortterm_portfolio.buy("DG", 29, 5505.65, datetime.date(2020, 7, 2))
# shortterm_portfolio.sell("PKI", 73, 7311.52, datetime.date(2020, 7, 7))
# shortterm_portfolio.buy("BLK", 13, 7161.18, datetime.date(2020, 7, 7))
# shortterm_portfolio.sell("ZBRA", 3, 793, datetime.date(2020, 7, 14))
# shortterm_portfolio.buy("DXCM", 2, 780, datetime.date(2020, 7, 14))
# shortterm_portfolio.sell("BLK", 13, 7623.66, datetime.date(2020, 7, 17))
# shortterm_portfolio.buy("MELI", 8, 7560.4, datetime.date(2020, 7, 17))
# shortterm_portfolio.deposit(4117.35, datetime.date(2020, 7, 20))
# shortterm_portfolio.buy("JD", 69, 4343.95, datetime.date(2020, 7, 20))
# shortterm_portfolio.sell("QRVO", 70, 8273.12, datetime.date(2020, 7, 21))

# shortterm_portfolio.deposit(280.48, datetime.date(2020, 7, 21))
# shortterm_portfolio.buy("JD", 135, 8553.6, datetime.date(2020, 7, 21))
# shortterm_portfolio.deposit(1235.4, datetime.date(2020, 7, 22))
# shortterm_portfolio.buy("JD", 20, 1235.4, datetime.date(2020, 7, 22))
# shortterm_portfolio.sell("DG", 29, 5591.66, datetime.date(2020, 7, 24))
# shortterm_portfolio.buy("SGEN", 33, 5566.44, datetime.date(2020, 7, 24))
# shortterm_portfolio.sell("MELI", 8, 8563.01, datetime.date(2020, 7, 27))
# shortterm_portfolio.deposit(22.47, datetime.date(2020, 7, 28))
# shortterm_portfolio.buy("BMRN", 70, 8610.7, datetime.date(2020, 7, 28))

In [ ]:
# longterm_portfolio.deposit(2024.7, datetime.date(2020, 6, 22))
# longterm_portfolio.buy("INTC", 34, 2024.7, datetime.date(2020, 6, 22))
# longterm_portfolio.deposit(4988.58, datetime.date(2020, 7, 8))
# longterm_portfolio.buy("OKE", 183, 4988.58, datetime.date(2020, 7, 8))
# longterm_portfolio.deposit(5993.46, datetime.date(2020, 7, 16))
# longterm_portfolio.buy("NLOK", 297, 5993.46, datetime.date(2020, 7, 16))
# longterm_portfolio.deposit(7486.72, datetime.date(2020, 7, 27))
# longterm_portfolio.buy("BMY", 128, 7486.72, datetime.date(2020, 7, 27))

In [ ]:
print(shortterm_portfolio.calc_rate_of_return())
print(shortterm_portfolio.calc_market_rate_of_return())
print(longterm_portfolio.calc_rate_of_return())
print(longterm_portfolio.calc_market_rate_of_return())

In [ ]:
shortterm_portfolio.save()
longterm_portfolio.save()

In [ ]:
df_dividends = {}

In [ ]:
f = open("./ticker.csv", "r")
tickers = f.read().split(",")

In [ ]:
%%time
f = open("./ticker.csv", "r")
tickers = f.read().split(",")
for t in tickers:
    stock = yf.Ticker(t)
    df_dividends[t] = stock.get_dividends()

In [ ]:
stock = yf.Ticker("MSFT")

In [ ]:
df_dividends

In [ ]:
stock = yf.download("AAPL")

In [ ]:
stock["Adj Close"][datetime.date(2014, 6, 5)]

In [ ]:
stock["Adj Close"][datetime.date.today()]

In [ ]:
a = yf.Ticker("AAPL")

In [ ]:
a.get_splits()

In [ ]:
8273.12-8553.6

In [ ]:
raw["DG"][datetime.date(2020, 7, 6)]

In [ ]:
raw.index.date == datetime.date(2020, 7, 6)

In [4]:
raw["Close"]["DG"][raw.index.get_loc(datetime.date(2020, 7, 6))-1]

189.47999572753906

In [ ]:
raw.Index

In [ ]:
f = open("./ticker.csv", "r")
tickers = f.read().split(",")
df = yf.download(" ".join(tickers), period="2y")

In [ ]:
a=df[["Adj Close", "Close"]]

In [7]:
raw["Close"].index.get_loc(datetime.date(2020, 7, 6))

487

In [9]:
raw.index

DatetimeIndex(['2018-07-30', '2018-07-31', '2018-08-01', '2018-08-02',
               '2018-08-03', '2018-08-06', '2018-08-07', '2018-08-08',
               '2018-08-09', '2018-08-10',
               ...
               '2020-07-15', '2020-07-16', '2020-07-17', '2020-07-20',
               '2020-07-21', '2020-07-22', '2020-07-23', '2020-07-24',
               '2020-07-27', '2020-07-28'],
              dtype='datetime64[ns]', name='Date', length=504, freq=None)

In [11]:
total_dividend = 10.44
num_shares = 29
ticker = "DG"
dividend_per_share = total_dividend / num_shares
pre_dividend_close = raw["Close"][ticker][raw.index.get_loc(datetime.date(2020, 7, 6))-1]
adj_factor = 1 - dividend_per_share / pre_dividend_close

In [13]:
adj_factor*pre_dividend_close

189.11999572753905

In [20]:
(1-.36/189.48)*192.14

191.77494616846104

0.014660005160321817